In [20]:
from __future__ import  print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [21]:
batch_size = 128
num_classes =10
epochs =12

# input image dimensions
img_row, img_cols = 28,28

(X_train, y_train), (X_test, y_test)= mnist.load_data()
print(X_train.shape)

(60000, 28, 28)


In [22]:
if K.image_data_format() == 'channels_first':
  X_train = X_train.reshape(X_train.shape[0],1, img_row, img_cols)
  X_test = X_test.reshape(X_test.shape[0],1, img_row, img_cols)
  input_shape = (1, img_row, img_cols)
else:
  X_train = X_train.reshape(X_train.shape[0], img_row, img_cols,1)
  X_test = X_test.reshape(X_test.shape[0],img_row, img_cols,1)
  input_shape = (img_row, img_cols,1 )

In [23]:
# Normalizing the pixel values
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /=255
X_test /=255
print('x_train shape:', X_train.shape)
print('x_test shape:', X_test.shape)

print(X_train.shape[0],'train samples')
print(X_test.shape[0],' test samples')


x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
60000 train samples
10000  test samples


In [24]:
# converting class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




### model 1: filter 3x3 
 a. basic model : without dropdout and batch norm

In [25]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
              

In [27]:
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_data=(X_test,y_test))   
              
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test loss',score[0])
print('Test accuracy', score[1])              

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 37s 623us/step - loss: 0.0603 - acc: 0.9822 - val_loss: 0.0553 - val_acc: 0.9820
Epoch 2/12
60000/60000 [==============================] - 38s 637us/step - loss: 0.0432 - acc: 0.9871 - val_loss: 0.0479 - val_acc: 0.9833
Epoch 3/12
60000/60000 [==============================] - 38s 631us/step - loss: 0.0309 - acc: 0.9910 - val_loss: 0.0488 - val_acc: 0.9845
Epoch 4/12
60000/60000 [==============================] - 38s 631us/step - loss: 0.0248 - acc: 0.9925 - val_loss: 0.0448 - val_acc: 0.9840
Epoch 5/12
60000/60000 [==============================] - 38s 633us/step - loss: 0.0175 - acc: 0.9951 - val_loss: 0.0454 - val_acc: 0.9857
Epoch 6/12
60000/60000 [==============================] - 39s 645us/step - loss: 0.0132 - acc: 0.9960 - val_loss: 0.0457 - val_acc: 0.9854
Epoch 7/12
60000/60000 [==============================] - 38s 626us/step - loss: 0.0107 - acc: 0.9970 - val_loss: 0.

b. Complex model with more layers
 - with dropdout and batch norm

In [28]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test,y_test))   
              
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test loss',score[0])
print('Test accuracy', score[1])              


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 184s 3ms/step - loss: 0.4028 - acc: 0.8763 - val_loss: 0.0651 - val_acc: 0.9781
Epoch 2/12
13824/60000 [=====>........................] - ETA: 2:16 - loss: 0.1571 - acc: 0.9517

KeyboardInterrupt: 

### model 2 :  filter 2x2
 a. basic model : without dropdout and batch norm

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test,y_test))   
              
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test loss',score[0])
print('Test accuracy', score[1])              

b. Complex model with more layers
- with dropdout and batch norm

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test,y_test))   
              
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test loss',score[0])
print('Test accuracy', score[1])              


### model 3: filter 5x5 
 a. basic model : without dropdout and batch norm

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test,y_test))   
              
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test loss',score[0])
print('Test accuracy', score[1])              

b. Complex model with more layers
- with dropdout and batch norm


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test,y_test))   
              
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test loss',score[0])
print('Test accuracy', score[1])              
